### Otimização dos Dados

- Continuando a etapa anterior, agora vamos avaliar os erros cometidos pelos modelos, buscando insights que nos guiem para alterar ou complementar os dados.

- Para avaliar os erros, vamos utilizar, primeiramente, a **matriz de confusão** da classificação.

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_iter=9, min_samples_leaf=5, min_samples_split=10, _)